# Nearest Regression with  `MPC dataset`

In [1]:
import torch
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r
import pickle
import mediapy as media

from policy import GaussianPolicy

np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
# %matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

/Users/yoonbyung/opt/anaconda3/envs/snapbot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MuJoCo version:[2.3.7]


In [2]:
with open(file='../data/qposnoise smpl stand v3.pkl', mode='rb') as f:
    dataset = pickle.load(f)

horizon = 300

action_batch = torch.Tensor(dataset['action'])
qpos_batch = torch.Tensor(dataset['qpos'])
qvel_batch = torch.Tensor(dataset['qvel'])

action_batch = action_batch.reshape(action_batch.shape[0] * horizon, -1)
qpos_batch = qpos_batch.reshape(qpos_batch.shape[0] * horizon, -1)
qvel_batch = qvel_batch.reshape(qvel_batch.shape[0] * horizon, -1)

del dataset

qpos_batch = qpos_batch[:, 2:]
# qpos_batch = torch.cat((qpos_batch[:-2, :],qpos_batch[1:-1, :],qpos_batch[2:, :]),dim=1)
# qvel_batch = torch.cat((qvel_batch[:-2, :],qvel_batch[1:-1, :],qvel_batch[2:, :]),dim=1)

obs_batch = torch.cat((qpos_batch, qvel_batch), dim=1)

# action_batch = action_batch[2:]
obs_dim = obs_batch.shape[1]
action_dim = action_batch.shape[1]
hidden_dim = 512

print("obs : ", obs_batch.shape)
print("action : ", action_batch.shape)


obs :  torch.Size([30000, 85])
action :  torch.Size([30000, 37])


In [3]:
xml_path = '../asset/smpl_rig/scene_smpl_rig.xml'
env = MuJoCoParserClass(name='smplrig',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

dt:[0.0020] HZ:[500]
n_dof (=nv):[43]
n_geom:[67]
geom_names:['floor', 'base', 'base2lpelvis', 'base2rpelvis', 'base2spine1', 'spine2spine', 'spine2spine2', 'spine2spine3', 'spine2lcollar', 'spine2rcollar', 'neck2head', 'head', 'nose', 'lcollar2lshoulder', 'lshoulder2lelbow', 'lelbow2lwrist', 'lwrist2lindex1', 'lwrist2lmiddle1', 'lwrist2lpinky1', 'lwrist2lring1', 'lwrist2lthumb1', 'lindex1-lindex2', 'lindex2-lindex3', 'lindex3-lindextip', 'lmiddle1-lmiddle2', 'lmiddle2-lmiddle3', 'lmiddl3-lmiddletip', 'lring1-lring2', 'lring2-lring3', 'lring3-lringtip', 'lpinky1-lpinky2', 'lpinky2-lpinky3', 'lpinky3-lpinkytip', 'lthumb1-lthumb2', 'lthumb2-lthumb3', 'lthumb3-lthumbtip', 'rcollar2rshoulder', 'rshoulder2relbow', 'relbow2rwrist', 'rwrist2rindex1', 'rwrist2rmiddle1', 'rwrist2rpinky1', 'rwrist2rring1', 'rwrist2rthumb1', 'rindex1-rindex2', 'rindex2-rindex3', 'rindex3-rindextip', 'rmiddle1-rmiddle2', 'rmiddle2-rmiddle3', 'rmiddle3-rmiddletip', 'rring1-rring2', 'rring2-rring3', 'rring3-rringtip

In [4]:
# Set which joints to control
ctrl_joint_names = env.ctrl_joint_names # <==
joint_idxs_fwd = env.get_idxs_fwd(joint_names=ctrl_joint_names)
joint_idxs_jac = env.get_idxs_jac(joint_names=ctrl_joint_names)
q_ctrl_init = env.get_qpos_joints(ctrl_joint_names)
n_ctrl_joint = len(ctrl_joint_names)

In [5]:
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='SMPL',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=False)
env.update_viewer(azimuth=152,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()

tick = 0
tick_inc = 0
rand_idx = torch.randperm(obs_batch.shape[0])
obs_batch = obs_batch[rand_idx]
image_list = []
while env.is_viewer_alive():
    qpos = env.data.qpos
    qvel = env.data.qvel
        
    state = np.concatenate((qpos[2:], qvel))
    state = torch.Tensor(state)
    # action, _ = policy(state)
    min_idx = torch.argmin(torch.mean(torch.abs(obs_batch - state), dim=1))
    print(min_idx)
    action = action_batch[min_idx]
    env.step(action.detach().numpy())
    
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(env.tick))
        # Plot foot
        # env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        # env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        # env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()

        if env.tick > 0 and env.tick % 1000 == 0:
            env.tick += 1
            
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

tensor(12495)
tensor(19767)
tensor(15679)
tensor(14685)
tensor(14685)
tensor(14685)
tensor(14685)
tensor(14685)
tensor(14685)
tensor(14685)
tensor(24777)
tensor(5872)
tensor(25760)
tensor(23489)
tensor(24476)
tensor(5514)
tensor(17713)
tensor(17713)
tensor(17713)
tensor(17713)
tensor(412)
tensor(412)
tensor(412)
tensor(412)
tensor(1873)
tensor(5515)
tensor(5515)
tensor(9453)
tensor(5836)
tensor(22824)
tensor(10967)
tensor(215)
tensor(215)
tensor(28026)
tensor(10449)
tensor(20941)
tensor(9757)
tensor(22410)
tensor(24897)
tensor(3915)
tensor(24897)
tensor(24897)
tensor(24897)
tensor(24897)
tensor(24897)
tensor(24897)
tensor(26850)
tensor(15892)
tensor(15892)
tensor(15892)
tensor(15892)
tensor(15892)
tensor(15892)
tensor(15892)
tensor(12142)
tensor(3568)
tensor(19759)
tensor(3745)
tensor(3568)
tensor(19759)
tensor(25723)
tensor(10771)
tensor(3817)
tensor(11861)
tensor(14121)
tensor(11513)
tensor(15411)
tensor(11513)
tensor(17721)
tensor(4116)
tensor(5109)
tensor(21433)
tensor(15686)
tenso

In [6]:
with media.VideoWriter(
    "x.mp4", shape=(480, 640), fps=5) as w:
    for image in image_list:
        w.add_image(image)

In [7]:
image_list[0].shape

IndexError: list index out of range